# **DASA CS-2: [TAS: Tele Assistance System]**

In [5]:
from Src.PyDASA.Measure.fdu import FDU
from Src.PyDASA.Measure.params import Parameter, Variable

## **Project Overview**

## **Software Architecture Summary**

### **Base Design (SOA)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

### **Arternative Design No.1 (Horizontal Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

### **Arternative Design No.2 (Vertical Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

## **Conclusion**

## **Future Work**